# Librairies Import

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import _pickle as cPickle

# Pre-processing Function

### To replace NaN values

In [59]:
#  mediane si quantative
#  mod si qualitative
cabin = df['Cabin'].unique()[5]
def remplaceNanValues(df, columnName) :
    if columnName == 'Age' :
        x = df[columnName].median()
    elif columnName == 'Cabin' :
        x = cabin
    else :
        x = df[columnName].unique()[0]
        
    for index, row in df.iterrows() :
        if pd.isna(row[columnName]):
            df.loc[index, columnName] = x        

In [43]:
def transformToString(df,columnName) :
    df[columnName] = df[columnName].apply(str)

In [44]:
def chooseAndDummifyVariables(df, features_columns, to_dummify, targetName):
    X = df[features_columns]
    y = df[targetName]    
    X = pd.get_dummies(X, columns = to_dummify)
    
    return X,y

# Model train, score and save Function

In [45]:
from sklearn.model_selection import train_test_split

#### To save the Model

In [46]:
def saveModel(model) :
    filename = ''+model+'.sav'
    cPickle.dump(model, open(filename, 'wb'))

#### Train and score the Model

In [47]:
def trainAndScore(model, X, y):
    liste_test_size = [a/20.0 for a in range(0,20)][1:]
    scores = []
    for ts in liste_test_size :
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = ts, random_state=0)
        clf = model.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        scores.append(score)
    print("[INFO] SCORES : ",scores)
    print("[INFO] MEAN = ",np.array(scores).mean())
    print("[INFO] MAX = ",max(scores))

# Importing Datas

In [143]:
df = pd.read_csv('data/train.csv', sep=',')

In [144]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [145]:
df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

### Specific Function

In [146]:
def getTitle(str):
    return str.split(',')[1].split('.')[0].split(' ')[1]

# Features selection

In [147]:
df['Pclass'].unique()

array([3, 1, 2], dtype=int64)

In [148]:
df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6], dtype=int64)

In [149]:
df['Sex'].unique()

array(['male', 'female'], dtype=object)

In [150]:
df['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

In [151]:
features_columns = ['Pclass', 'Parch', 'Sex', 'Age', 'SibSp', 'Fare']
to_dummify = ['Pclass', 'Parch', 'Sex', 'SibSp']
targetName = 'Survived'

In [153]:
remplaceNanValues(df, 'Age')
remplaceNanValues(df, 'Cabin')
remplaceNanValues(df, 'Embarked')

In [154]:
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [155]:
X, y = chooseAndDummifyVariables(df, features_columns, to_dummify, targetName)

In [156]:
X.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,...,Parch_6,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,22.0,7.2500,0,0,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1,38.0,71.2833,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,26.0,7.9250,0,0,1,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,35.0,53.1000,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,35.0,8.0500,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


# Features Engineering

In [157]:
X['is_child'] = X['Age'] <= 11

In [158]:
X['Title'] = df['Name'].apply(lambda x : getTitle(x) )

In [159]:
X['Surname'] = df['Name'].apply(lambda x : '(' in x)

In [161]:
X = pd.get_dummies(X, columns=['Title'])

In [162]:
X.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the
0,22.0,7.2500,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,38.0,71.2833,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,26.0,7.9250,0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,35.0,53.1000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,35.0,8.0500,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# Model build

## SVM

In [88]:
from sklearn.svm import SVC

In [170]:
model = SVC(gamma='auto')

In [171]:
trainAndScore(model, X, y)

[INFO] SCORES :  [0.7777777777777778, 0.7333333333333333, 0.7611940298507462, 0.7653631284916201, 0.7668161434977578, 0.7649253731343284, 0.7467948717948718, 0.7366946778711485, 0.7281795511221946, 0.7242152466367713, 0.7331975560081466, 0.7214953271028037, 0.7120689655172414, 0.7099358974358975, 0.6920777279521674, 0.6605890603085554, 0.6701846965699209, 0.6633416458852868, 0.5879574970484062]
[INFO] MEAN =  0.718744342491525
[INFO] MAX =  0.7777777777777778
